In [65]:
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [66]:
%pip install numpy as np

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [67]:
import tensorflow as tf
from tensorflow import keras

In [68]:
%pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


In [69]:
import cv2
import os
import numpy as np

In [70]:
project_path=r"C:\Users\Asus\Desktop\NeuroDoku"
sudoku_images_digits=os.path.join(project_path,r"digits_updated\digits_updated")


In [71]:
sudoku_images_digits

'C:\\Users\\Asus\\Desktop\\NeuroDoku\\digits_updated\\digits_updated'

In [72]:
data_list=os.listdir(sudoku_images_digits)
print(data_list)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [73]:
data_list

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [107]:
data_X=[]
data_Y=[]

In [92]:
print(type(data_list))
print(os.listdir(os.path.join(sudoku_images_digits,r"0")))

<class 'list'>
['img001-00001.png', 'img001-00002.png', 'img001-00003.png', 'img001-00004.png', 'img001-00005.png', 'img001-00006.png', 'img001-00007.png', 'img001-00008.png', 'img001-00009.png', 'img001-00010.png', 'img001-00011.png', 'img001-00012.png', 'img001-00013.png', 'img001-00014.png', 'img001-00015.png', 'img001-00016.png', 'img001-00017.png', 'img001-00018.png', 'img001-00019.png', 'img001-00020.png', 'img001-00021.png', 'img001-00022.png', 'img001-00023.png', 'img001-00024.png', 'img001-00025.png', 'img001-00026.png', 'img001-00027.png', 'img001-00028.png', 'img001-00029.png', 'img001-00030.png', 'img001-00031.png', 'img001-00032.png', 'img001-00033.png', 'img001-00034.png', 'img001-00035.png', 'img001-00036.png', 'img001-00037.png', 'img001-00038.png', 'img001-00039.png', 'img001-00040.png', 'img001-00041.png', 'img001-00042.png', 'img001-00043.png', 'img001-00044.png', 'img001-00045.png', 'img001-00046.png', 'img001-00047.png', 'img001-00048.png', 'img001-00049.png', 'img

In [108]:
for image_name in data_list:
    for digit_name in os.listdir(os.path.join(sudoku_images_digits,f'{image_name}')):
        image_path_folder=os.path.join(sudoku_images_digits,f"{image_name}")
        image_path=os.path.join(image_path_folder,digit_name)
        pic=cv2.imread(image_path)
        pic=cv2.resize(pic,(32,32))
        data_X.append(pic)
        label=int(image_name)
        data_Y.append(label)


In [109]:
len(data_Y)

10160

In [110]:
if len(data_X)==len(data_Y):
    print("Toatl Data Points :", len(data_Y))

Toatl Data Points : 10160


In [111]:
data_X


[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [112]:
data_X=np.array(data_X)
data_Y=np.array(data_Y)

In [113]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# train_y_categorical=to_categorical(data_Y,np.max(data_Y)+1)

In [82]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [114]:
import sklearn

In [115]:
from sklearn.model_selection import train_test_split

In [116]:
train_X, test_X, train_y, test_y = train_test_split(data_X,data_Y,test_size=0.05)
train_X, valid_X, train_y, valid_y = train_test_split(train_X,train_y,test_size=0.2)
print("Training Set Shape = ",train_X.shape)
print("Validation Set Shape = ",valid_X.shape)
print("Test Set Shape = ",test_X.shape)

Training Set Shape =  (7721, 32, 32, 3)
Validation Set Shape =  (1931, 32, 32, 3)
Test Set Shape =  (508, 32, 32, 3)


In [117]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

In [118]:
def Prep(img):
    if len(img.shape) > 2 and img.shape[2] > 1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert to grayscale
    img = img.astype(np.uint8) # convert to 8-bit unsigned integer
    img = cv2.equalizeHist(img) # histogram equalization
    img = img/255 # normalizing
    return img

train_X = np.array(list(map(Prep, train_X)))
test_X = np.array(list(map(Prep, test_X)))
valid_X= np.array(list(map(Prep, valid_X)))

#Reshaping the images
train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], train_X.shape[2],1)
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], test_X.shape[2],1)
valid_X = valid_X.reshape(valid_X.shape[0], valid_X.shape[1], valid_X.shape[2],1)

#Augmentation
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.2, shear_range=0.1, rotation_range=10)
datagen.fit(train_X)

In [119]:
data_classes = np.max(data_Y) + 1
train_y = to_categorical(train_y, data_classes)
test_y = to_categorical(test_y, data_classes)
valid_y = to_categorical(valid_y, data_classes)

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D


In [121]:
model = Sequential()

model.add((Conv2D(60,(5,5),input_shape=(32, 32, 1) ,padding = 'Same' ,activation='relu')))
model.add((Conv2D(60, (5,5),padding="same",activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add((Conv2D(30, (3,3),padding="same", activation='relu')))
model.add((Conv2D(30, (3,3), padding="same", activation='relu')))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 60)     │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 60)     │        90,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 30)     │        16,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 30)     │         8,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1920)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 500)            │       960,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,081,490 (4.13 MB)

 Trainable params: 1,081,490 (4.13 MB)

 Non-trainable params: 0 (0.00 B)

In [122]:
from tensorflow.keras.optimizers import RMSprop

In [124]:
train_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [125]:
optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon = 1e-08, decay=0.0)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

train_y_int = np.argmax(train_y, axis=1)
valid_y_int = np.argmax(valid_y, axis=1)

history = model.fit(train_X, train_y_int, 
                    epochs=30, 
                    validation_data=(valid_X, valid_y_int), 
                    verbose=2, 
                    batch_size=32)

Epoch 1/30


c:\Users\Asus\anaconda3\envs\sudoku_solver\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


242/242 - 11s - 45ms/step - accuracy: 0.8246 - loss: 0.5184 - val_accuracy: 0.9715 - val_loss: 0.0773
Epoch 2/30
242/242 - 11s - 45ms/step - accuracy: 0.9663 - loss: 0.1073 - val_accuracy: 0.9834 - val_loss: 0.0523
Epoch 3/30
242/242 - 12s - 50ms/step - accuracy: 0.9773 - loss: 0.0695 - val_accuracy: 0.9850 - val_loss: 0.0479
Epoch 4/30
242/242 - 13s - 55ms/step - accuracy: 0.9842 - loss: 0.0556 - val_accuracy: 0.9959 - val_loss: 0.0197
Epoch 5/30
242/242 - 21s - 88ms/step - accuracy: 0.9859 - loss: 0.0467 - val_accuracy: 0.9902 - val_loss: 0.0259
Epoch 6/30
242/242 - 30s - 123ms/step - accuracy: 0.9891 - loss: 0.0327 - val_accuracy: 0.9855 - val_loss: 0.0661
Epoch 7/30
242/242 - 11s - 45ms/step - accuracy: 0.9898 - loss: 0.0334 - val_accuracy: 0.9953 - val_loss: 0.0169
Epoch 8/30
242/242 - 11s - 45ms/step - accuracy: 0.9909 - loss: 0.0344 - val_accuracy: 0.9959 - val_loss: 0.0129
Epoch 9/30
242/242 - 11s - 45ms/step - accuracy: 0.9924 - loss: 0.0273 - val_accuracy: 0.9896 - val_loss: 